<a href="https://colab.research.google.com/github/anhnguyenvv/NLP-A/blob/main/project/ai_ali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install gradio -q
!pip install openai -q
!pip install gtts -q
!pip install pydub -q
!pip install config -q
!pip install git+https://github.com/openai/whisper.git -q
!sudo apt update && sudo apt install ffmpeg
!pip install gdown -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 229 kB in 2s (94.2 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state in

In [6]:
import gradio
import config
from gtts import gTTS
import os
import subprocess
from pydub import AudioSegment
import math
import whisper
from openai import OpenAI
import requests, gdown
import datetime

### load whisper model

In [7]:
import torch

# Checking if NVIDIA GPU is available
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
W_model = whisper.load_model("base", device=DEVICE)

In [8]:
#File Project1_Data.json
file_id = ["1v7SsfHr-slNIjVZ7tnliNJXYetVgwVya",
           "1vtdKzRvIwINdWEmnKG-U-VhFkrzTLDRL",
           "1v7SsfHr-slNIjVZ7tnliNJXYetVgwVya"
]
datafilename =[ "./Samples/SampleCall1.mp3",
        "./Samples/SampleCall2.mp3",
        "./Samples/SampleCall3.mp3"]
import os
os.makedirs('Samples')
for i in range(len(file_id)):
   with open(datafilename[i], 'wb') as f:
     gdown.download(f"https://drive.google.com/uc?id={file_id[i]}", datafilename[i])

Downloading...
From: https://drive.google.com/uc?id=1v7SsfHr-slNIjVZ7tnliNJXYetVgwVya
To: /content/Samples/SampleCall1.mp3
100%|██████████| 2.86M/2.86M [00:00<00:00, 188MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vtdKzRvIwINdWEmnKG-U-VhFkrzTLDRL
To: /content/Samples/SampleCall2.mp3
100%|██████████| 1.85M/1.85M [00:00<00:00, 148MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v7SsfHr-slNIjVZ7tnliNJXYetVgwVya
To: /content/Samples/SampleCall3.mp3
100%|██████████| 2.86M/2.86M [00:00<00:00, 144MB/s]


In [9]:
tran = W_model.transcribe("./Samples/SampleCall2.mp3")


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [10]:
tran

{'text': " Thank you for calling Martha's Flores, Thomas Cisteri. Hello, I'd like to order flowers and I think you have what I'm looking for. I'd be happy to take care of your order. May I have your name please? Randall Thomas. Randall Thomas, can you spell that for me? Randall, AN, B-A-L-L. Thomas, D-H-O, M-A. Thank you for that information Randall. May I have your home or office number area code first? Erickert 409, then 5-866-5088. That's 409-866-5088. Do you have a fax number or email address? Email is vandal.com and gmail.com. Randall.Thomasatgmail.com may have your shipping address? 6800. Okay. Glad it's Avenue, Beaumont, Texas, Zip Code, 777-706. Glad it's Avenue, Beaumont, Texas, Zip Code, 777-706. Thank you for the information. What products were you interested in purchasing? Red roses, probably a dozen. One dozen of red roses? Do you want long stems? Sure. All right, Randall, let me process your order. One moment please. Okay. Randall, you're ordering one dozen long-stent red

In [11]:
# Set your OpenAI API key
client = OpenAI(api_key = "sk-oGx8ZhLZ0xMahXqAF3tGcsTQhfO7WGIiRC0UlwtNYKJU9pxu",
                base_url="https://api.chatanywhere.tech/v1")
messages_system = {}
messages_system['en']= {"role": "system",
                        "content": "You are a call center quality and assurance auditor. Your job is to review the call recording, and provide a very brief summary of the key information in the call including Operator’s Name, Call Category, Issue, and Solution. Also, you need to conduct sentiment analysis on the call and evaluate the customers satisfaction rate from 1 to 10 and provide a very short straight-to-the-point area of improvement to the operator."}
messages_system['vi']= {"role": "system",
                        "content": "Bạn là kiểm toán viên đảm bảo chất lượng của trung tâm chăm sóc khách hàng. Công việc của bạn là xem lại bản ghi cuộc gọi và cung cấp bản tóm tắt rất ngắn gọn về thông tin chính trong cuộc gọi bao gồm: Tên người trực tổng đài, Loại cuộc gọi, Vấn đề, và Giải pháp. Ngoài ra, bạn cần tiến hành phân tích cảm xúc của khách hàng trong cuộc gọi và đánh giá mức độ hài lòng của khách hàng từ 1 đến 10, đồng thời cung cấp một phạm vi cải tiến rất ngắn gọn và đi thẳng vào vấn đề cho người trực tổng đài."}
# Transcribe audio using OpenAI's Whisper
def transcribe_audio(audio):
    segment_length = 600000
    # Open the audio file
    audio_file = AudioSegment.from_file(audio)
    # Get the duration of the audio file in milliseconds
    duration_ms = len(audio_file)
    # Calculate the number of segments needed
    num_segments = math.ceil(duration_ms / segment_length)
    # Create an empty string to hold the concatenated text
    all_text = {'text': "",
                'segments': ""}
    # Split the audio file into segments
    for i in range(num_segments):
        start = i * segment_length
        end = min((i + 1) * segment_length, duration_ms)
        segment = audio_file[start:end]
        segment.export(f"segment_{i}.mp3", format="mp3")


    for i in range(num_segments):
        audio_file = open(f"segment_{i}.mp3", "rb")
        transcript = W_model.transcribe(f"segment_{i}.mp3")
        all_text['text'] += transcript['text']
        for s in transcript['segments']:
          all_text['segments'] += str(datetime.timedelta(seconds = s['start'] + i*600.0)) + " - " + str(
              datetime.timedelta(seconds = s['end'] + i*600.0)) + ": " + s['text'] +"\n"

    return all_text

# Analyze transcript using OpenAI's GPT-3
def analyze_transcript(transcript, language= 'en'):
    global messages_system
    messages=[]
    messages.append(messages_system[language])
    messages.append({"role": "user", "content": transcript['text']})
    response = client.chat.completions.create(
          messages= messages,
          model="gpt-3.5-turbo",
      )

    systems_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": systems_message})

    chat_transcript = ""
    for message in messages:
        if message['role'] != 'system':
            chat_transcript += message['role'] + ": " + message['content'] + "\n\n"
    result = "Transcript: " + transcript['segments'] + "\n\n"
    '''for s in transcript['segments']:
      result += str(datetime.timedelta(seconds = s['start'])) + " - " + str(
          datetime.timedelta(seconds = s['end'])) + ": " + s['text'] +"\n"'''
    return systems_message +"\n\n"+ result
# Define the transcription and analysis function
def transcribe_and_analyze(mic=None, file=None, language='en'):
    if mic is not None:
        audio = mic
    elif file is not None:
        audio = file
    else:
        return "You must either provide a mic recording or a file"
    # Transcribe the audio to text
    #transcript =  "Thank you for calling Coats and Gowns, my name is Sam, how can I help you? Oh yes, I bought a coat from you guys but I need to return it because it was the wrong size and my size is not available. I haven't received any email from you and I wonder if my payment has been refunded. Seems like it does not apply on my account yet. I do apologize for the inconvenience but let me go ahead and check if there are some notes in the order details. May I ask for the original online order number? Okay, let me see. Please wait for a moment. Okay, hold on. Okay, I believe it's 017-1425-793. Thank you. Just to verify that's 017-1425-793, correct? Yes, that's right. I do apologize but the one that you provided is actually not accepting towards our database. Do you actually have a number that starts with 0007 or 7? Or can I ask for your first and last name so I can manually check it in our database? Yeah, I'm not sure about 000 something. Anyway, it's Adam Wilson. Alright, thank you. So Wilson is spelled as W for Whiskey, I for India, L for Lima, S for Sierra, O for Oscar and for November. Is that correct, Adam? Yes. Alright, thank you for that information. Okay, let me just quickly log into a portal and check any notes in your order. Do you still have the return tracking from the courier company when you return the item? Not sure. Is it the 74391? That's actually the return authorization from coats and gowns. When you return the item to the courier company for shipping, they should have provided you with the return tracking. Do you still have that? It's actually printed in the upper right part of the receipt. Okay, I probably have to dig it up. Okay, I imagine I have it somewhere but it might take some time to look for that. Oh, I see. Anyway, I tried to search for your name and nothing is coming up. Can I ask for the order under your name originally? Yeah, yeah, it's under my name. So it's not under Andy or something? It's Adam? Yeah, exactly. Thank you. Now, what's the phone number that you put in the order before? Okay, probably it's 411-345-0377. Thank you. So it's 411-345-0377, is that correct? Oh, it's 0377. Okay, and what's the email address that you gave when you filled out the order form? Okay, that should be adam.wilson.mailfence.com. Thank you. So again, it's adam.wilson.mailfence.com, is that correct? Yes, it is. Thank you. Adam, Adam underscore Wilson at mailfence.com. Okay, thank you. Now, I just want to make sure that I have the correct details. Please hold on for a moment while we check on the details here in my end, okay? Oh, sure, no problem. All right. Okay, thank you for patiently waiting, Adam. Now, I can actually see here that you have called to inform us that you'd like to return the item on the 15th of November, is that correct? Yes. Now, did you return the item to the courier company on the same day that you called? Probably close to it. I'm not really sure if it's on the same day though, but probably just close to that day. I understand. However, upon checking the date that you actually called in for the return, it's already beyond the timeframe that we ask for refunds. So, what we'll do is I'll go ahead and forward this information to our corporate office and I'll have them check and update you directly about the refund. So, please expect an email within two to four business days. Is that okay with you? Two to four business days? Yes. No worries. I will send it to them right now. Oh really? Okay. Alright, alright. At first I thought that it will probably be like three to five weeks to process. Oh, no, no, no, no. Don't worry, Adam. It's only two to four business days. So, you already checked your account and nothing has been credited yet on the card ending in 7-5-3, is that right? Nothing yet. Okay, I understand. And I'll go ahead and forward them to the... forward them the details that I have here. So, I'll send the email to adam.wilson.mailfence.com, correct? Yes, that's right. Alright, thank you so much for that information, Adam. I will be forwarding this over to our corporate office now and thank you again for calling Coats and Gowns. Is there anything else I can help you with today? Oh, no, that's all. Thank you. You're welcome and my pleasure. Have a nice day. Bye. Alright, thank you. Thank you."
    transcript = transcribe_audio(audio)
    # Analyze sentiment and summarize
    analysis = analyze_transcript(transcript, language)
    return analysis



Demo by gradio Interface

In [12]:

io = gradio.Interface(fn=transcribe_and_analyze,
                      inputs=[
                          gradio.Audio(sources="microphone", type="numpy", label="Speak here..."),
                          gradio.Audio(sources="upload", type = "filepath"),
                          gradio.Radio(["en", "vi"], label="Language")
                      ],
                      outputs="text",
                      title="AI Auditor for Call Center's Quality Assurance",
                      description="AI Alliance for Audio Analytics Team. Our project's objective is to conduct quality assurance on recorded calls, by transcribing the speech in the call to text using Whisper and then employing GPT-3 for sentiment analysis, summarisation, and feedback including areas for improvement. ",
                      examples=[[None, "./Samples/SampleCall1.mp3"],
                                [None, "./Samples/SampleCall2.mp3"],
                                [None, "./Samples/SampleCall3.mp3"]],
                      )

io.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://84826ec6472e889f31.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
